# Data Pre processing

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
#TF 2.3 
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

2.6.0


In [3]:
import tensorflow as tf
#TF 2.3 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#print(tf.__version__)

Num GPUs Available:  1


In [4]:
import os
#print(os.getcwd()+)

In [5]:
#!pip install openpyxl
#!pip install matplotlib
#!pip install transformers
#!pip install sklearn

In [6]:
import pandas as pd
import matplotlib as plt

In [7]:
#df = pd.read_csv('data\train.tsv', sep='\t')
df = pd.read_excel("data\Entity_sentiment_trainV2.xlsx")#pd.read_excel('data\Entity_sentiment_trainV2.xlsx', sheetname=None)
df.head()

Sentence      Entity Sentiment
0  The website was very easy to use and my insura...     website  positive
1  The web sight was easy to understand and I got...   web sight  positive
2  Having filled in the application on-line I cou...       point  negative
3  After finding AXA was cheaper than my renewal ...      prices  positive
4  The quote was a reasonable price compared with...  insurances  positive

In [8]:
import unicodedata
unicodedata.normalize('NFC', 'Quite refreshing from my other insurance company and also a Â£300 save.') == 'Quite refreshing from my other insurance company and also a A£300 save.'
#unicodedata.normalize('NFKD', 'Quite refreshing from my other insurance company and also a Â£300 save.') == 'Quite refreshing from my other insurance company and also a A£300 save.'

False

In [9]:
#unicodedata.normalize('NFKD', 'Quite refreshing from my other insurance company and also a Â£300 save.')
#Ⓣⓗⓐⓝⓚⓢ ⓕⓞⓡ ⓡⓔⓐⓓⓘⓝⓖ
unicodedata.normalize('NFKD', 'Ⓣⓗⓐⓝⓚⓢ ⓕⓞⓡ ⓡⓔⓐⓓⓘⓝⓖ')

'Thanks for reading'

In [10]:
binary_mapping = {'negative': 0,
 'positive': 1}


df['Sentiment'] = df['Sentiment'].map(binary_mapping)
df.head()

Sentence      Entity  Sentiment
0  The website was very easy to use and my insura...     website          1
1  The web sight was easy to understand and I got...   web sight          1
2  Having filled in the application on-line I cou...       point          0
3  After finding AXA was cheaper than my renewal ...      prices          1
4  The quote was a reasonable price compared with...  insurances          1

In [11]:


import numpy as np

seq_len = 512
num_samples = len(df)

num_samples, seq_len



(5999, 512)

In [12]:
from transformers import BertTokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
n=5
seed = 4568
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

'''
# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# tokenize - this time returning Numpy tensors
tokens = tokenizer(df['Sentence'].tolist(), max_length=seq_len, truncation=True,
                   padding='max_length', add_special_tokens=True,
                   return_tensors='np')
                   
'''

"\n# initialize tokenizer\ntokenizer = BertTokenizer.from_pretrained('bert-base-cased')\n\n# tokenize - this time returning Numpy tensors\ntokens = tokenizer(df['Sentence'].tolist(), max_length=seq_len, truncation=True,\n                   padding='max_length', add_special_tokens=True,\n                   return_tensors='np')\n                   \n"

In [13]:
def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

def map_func(input_ids, masks, entities, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'entities':entities, 'attention_mask': masks}, labels

In [14]:
class F1Score(tf.keras.metrics.Metric):

    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.bool)
        y_pred = tf.cast(y_pred, tf.bool)

        values = tf.logical_and(tf.equal(y_true, True), tf.equal(y_pred, True))
        values = tf.cast(values, self.dtype)
        if sample_weight is not None:
            sample_weight = tf.cast(sample_weight, self.dtype)
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))

    def result(self):
        return self.true_positives

    def reset_states(self):
        self.true_positives.assign(0)

m = F1Score()
m.update_state([0, 1, 1, 1], [0, 1, 0, 0])
print('Intermediate result:', float(m.result()))

m.update_state([1, 1, 1, 1], [0, 1, 1, 0])
print('Final result:', float(m.result()))

Intermediate result: 1.0
Final result: 3.0


In [15]:
def get_f1_score(precision, recall):
    f1 = ((precision*recall)/(precision+recall))*2
    return f1

In [16]:
def get_model_name(k):
    return '\model_'+str(k)+'.h5'

In [17]:
#!pip install tensorflow_addons

In [18]:
from transformers import TFAutoModel
import tensorflow as tf
import tensorflow_addons as tfa
def create_Model():
    
    bert = TFAutoModel.from_pretrained('bert-base-cased')
    #bert = TFAutoModel.from_pretrained('princeton-nlp/sup-simcse-bert-base-uncased')#princeton-nlp/unsup-simcse-bert-base-uncased')
    #bert = TFAutoModel.from_pretrained('princeton-nlp/unsup-simcse-bert-base-uncased')
    # we can view the model using the summary method
    bert.summary()
    
    # two input layers, we ensure layer name variables match to dictionary keys in TF dataset
    input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
    mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')
    #entities = tf.keras.layers.Input(shape=(512,), name='entities', dtype='int32')

    
    # we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
    embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access final activations (alread max-pooled) [1]
    #embeddings = bert.bert(input_ids, entities, attention_mask=mask )
    # convert bert embeddings into 2 output classes
    x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
    y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)
    
    
    model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

    # (optional) freeze bert layer
    model.layers[2].trainable = False

    # print out model summary
    model.summary()

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
    loss = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
    f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5)
    #f1  = tf.keras.metrics.CategoricalAccuracy('f1') num_classes=1, threshold=0.5
    recall = tf.keras.metrics.Recall()
    precision = tf.keras.metrics.Precision()
    #model.add

    model.compile(optimizer=optimizer, loss=loss, metrics=[acc, recall, precision])
    
    return model

In [19]:
from transformers import AutoTokenizer
from datetime import datetime
from packaging import version

#import tensorflow as tf
from tensorflow import keras

logdir = str.format('logs\scalars\{0}', datetime.now().strftime("%Y%m%d-%H%M%S"))

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
#tokenizer = AutoTokenizer.from_pretrained('princeton-nlp/sup-simcse-bert-base-uncased')
#tokenizer = AutoTokenizer.from_pretrained('princeton-nlp/unsup-simcse-bert-base-uncased')
#tokenizer.encode(add_special_tokens=True, text=)

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []
batch_size = 16
EPOCHS = 3

save_dir = os.getcwd()+'\saved_models'
fold_var = 1
Y = df[['Sentiment']]
#X = df['Sentence']
X = df['Sentence'] +" : "+ df['Entity']
for train_index, val_index in skf.split(X,Y):
    training_data = X.iloc[train_index]
    validation_data = X.iloc[val_index]
    #print(train_index + '| '+ training_data + ' | |'+ val_index + '|' +validation_data)
    #print(str.format('{0} | {1} || {2} | {3}', train_index, training_data, val_index, validation_data))
    
    #-------------------------------------------------Train DataSet ----------------------------------------------------#
    num_samples = len(training_data)
    ###arr = validation_data.values
    
    train_lbl =  Y.iloc[train_index]
    train_arr = np.array(train_lbl['Sentiment'].values.tolist())

    train_labels = np.zeros((num_samples, train_arr.max()+1))

    
    train_labels[np.arange(num_samples), train_arr] = 1


    # tokenize - this time returning Numpy tensors
    
    train_tokens = tokenizer(training_data.tolist(), max_length=seq_len, truncation=True,
                   padding='max_length', add_special_tokens=True,
                   return_tensors='np') 
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train_tokens['input_ids'], 
                                                        train_tokens['attention_mask'], 
                                                        train_labels))
    train_dataset = train_dataset.map(map_func)
    #dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
  

    print(train_dataset.take(1))
    #-------------------------------------------------Validation DataSet ----------------------------------------------------#
    num_samples = len(validation_data)
    ###arr = validation_data.values
    
    val_lbl =  Y.iloc[val_index]
    val_arr = np.array(val_lbl['Sentiment'].values.tolist())
    ###arr = lbl['Sentiment']
    #print(type(arr))
    
    #print(num_samples)
    #print(arr)
    val_labels = np.zeros((num_samples, val_arr.max()+1))
    #print(labels.shape)
    
    val_labels[np.arange(num_samples), val_arr] = 1

    #print(labels)

    # tokenize - this time returning Numpy tensors
    val_tokens = tokenizer(validation_data.tolist(), max_length=seq_len, truncation=True,
                   padding='max_length', add_special_tokens=True,
                   return_tensors='np')
    validation_dataset = tf.data.Dataset.from_tensor_slices((val_tokens['input_ids'], 
                                                             val_tokens['attention_mask'], 
                                                             val_labels))
    validation_dataset = validation_dataset.map(map_func)
    validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True)
    #batch_size = 16

    print(validation_dataset.take(1))

    model = create_Model()

    
    # CREATE CALLBACKS
    '''
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                            monitor='val_accuracy', verbose=1, 
                            save_best_only=True, mode='max')
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
    
    callbacks_list = [checkpoint, tensorboard_callback]
    
    history = model.fit(train_dataset, validation_data=validation_dataset, epochs=3, callbacks=callbacks_list, verbose=1)
    
    model.load_weights(save_dir +"\model_"+str(fold_var)+".h5")
    '''
    #------------------------------------------------------------------------
    
    checkpoint_filepath = save_dir + get_model_name(fold_var)
    #model.load_weights(checkpoint_filepath)
    print(get_model_name(fold_var))
    print(checkpoint_filepath)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                   save_weights_only=False,
                                                                   monitor='val_accuracy',
                                                                   mode='max',
                                                                   verbose=1,
                                                                   save_best_only=True)
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
    earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

    # Model weights are saved at the end of every epoch, if it's the best seen
    # so far.
    model.fit(train_dataset, 
              validation_data=validation_dataset,
              epochs=EPOCHS, 
              callbacks=[model_checkpoint_callback, tensorboard_callback, earlyStopping], 
              verbose=1)

    # The model weights (that are considered the best) are loaded into the model.
    model.load_weights(save_dir +"\model_"+str(fold_var)+".h5")
    #----------------------------------------------------------------------------
    #results = model.evaluate(validation_data)
    #results = dict(zip(model.metrics_names,results))
    
    #VALIDATION_ACCURACY.append(results['accuracy'])
    #VALIDATION_LOSS.append(results['loss'])
    
    fold_var +=1
    tf.keras.backend.clear_session()

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 2)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>
<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 2)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>


Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)  

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs/scalars

with open('review-labels.npy', 'wb') as f:
    np.save(f, labels)

# Input Pipeline


In [ ]:
#!pip install tensorflow

batch_size = 16

dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

dataset.take(1)

from sklearn.model_selection import StratifiedKFold
import numpy as np

x = np.random.randn(15, 3)
y = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=28)
for train_index, test_index in k_fold.split(dataset, y):
    print(f"Train index: {train_index} Test index: {test_index}")

from sklearn.model_selection import StratifiedKFold
import numpy as np

#x = np.random.randn(15, 3)
#y = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=28)
for train_index, test_index in k_fold.split(dataset, y):
    print(f"Train index: {train_index} Test index: {test_index}")